In [2]:
import numpy as np
import pandas as pd
import sklearn
import time
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from pprint import pprint
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from textblob.classifiers import NaiveBayesClassifier

from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, chi2

In [3]:
# Properties
py_file = "LinearSCVResults.py"
date_file = "data/tweets_all.csv"
start_time = time.time()
testd_size = 0.35
sarcastic_tweets = []
normal_tweet = []
stopNL = ['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere']

In [4]:
# Read Data Set
df = pd.read_csv(date_file, delimiter='|', encoding="utf-8", quotechar='"', header=None, names=['tweetID','tweet', 'target','pos','neu','neg','caps'])
df = df.dropna()
df.head()
df["target"] = df["target"].convert_objects(convert_numeric=True)
print(df.isnull().sum())
df = df.dropna()

df["target"] = df["target"].astype(int)
print(df.isnull().sum())

tweetID    0
tweet      0
target     0
pos        0
neu        0
neg        0
caps       0
dtype: int64
tweetID    0
tweet      0
target     0
pos        0
neu        0
neg        0
caps       0
dtype: int64


In [49]:
# df['f1'] = pd.Series(np.random.uniform(0,15,len(df)), index=df.index)
df.head()

,tweetID,tweet,target,pos,neu,neg,caps
0,500370339566985216,Zo fijn die #PostNL beloofd eerst tm vanmorgen...,1,0.091,0.909,0.000,0
1,411900187649794048,Heel fijn Vast op Dordrecht geen treinverkeer ...,1,0.114,0.762,0.124,0
2,175627549802639361,echt geweldig blyk denk dat ik dan maar weer v...,1,0.186,0.814,0.000,0
3,321699315707936769,fijn een product hebben dat niet werkt #kpn,1,0.184,0.816,0.000,0
4,258575794601865217,lekker chatten met een medewerker van tmobile,1,0.327,0.673,0.000,0


In [346]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df[['tweet','pos','neu','neg','caps']]
, df['target'], test_size=0.25)

In [347]:
from sklearn.base import TransformerMixin
class ColumnExtractor(TransformerMixin):

    def __init__(self, columns=[]):
        self.columns = columns

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        #print(self.transform(X))
        return self.transform(X)

    def transform(self, X, **transform_params):
        #print(X[self.columns])
        return X[self.columns]

    def fit(self, X, y=None, **fit_params):
        #print(self)
        return self

In [348]:
class Reshaper(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        #print(X)
        return X.reshape((len(X), 1))

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        #print(self.transform(X))
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        #print(self)
        return self

In [349]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from scikit_helpers.features import FeatureNamePipeline

pipe = Pipeline([
    ('features', FeatureUnion
     ([      
        ('pos', Pipeline([
            ('extract', ColumnExtractor('pos')),
            ('to_dense', Reshaper())
        ])),
        ('neu', Pipeline([
            ('extract', ColumnExtractor('neu')),
            ('to_dense', Reshaper())
        ])),
        ('neg', Pipeline([
            ('extract', ColumnExtractor('neg')),
            ('to_dense', Reshaper())
        ])),
#         ('caps', Pipeline([
#             ('extract', ColumnExtractor('caps')),
#             ('to_dense', Reshaper())
#         ])),
        ('tweet', Pipeline([
            ('extract', ColumnExtractor('tweet')),
            ('vect', CountVectorizer(ngram_range=(1, 3), max_df=0.3, min_df=0.0001, max_features=10000)),
            ('tfidf', TfidfTransformer(sublinear_tf= True, use_idf=False)),
            ("kbest", SelectKBest(chi2,k=700))
            ]))      
        ])),
#         ("kbest", SelectKBest(chi2,k=600)),
        ('clf', MultinomialNB(alpha=1e-05))])
pipe.fit(X_train, y_train)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('pos', Pipeline(steps=[('extract', <__main__.ColumnExtractor object at 0x0000000029D91940>), ('to_dense', <__main__.Reshaper object at 0x0000000029D91A58>)])), ('neu', Pipeline(steps=[('extract', <__main__.ColumnExtractor object at...  transformer_weights=None)), ('clf', MultinomialNB(alpha=1e-05, class_prior=None, fit_prior=True))])

In [350]:
y_pred = pipe.predict(X_test)

In [351]:
from sklearn.metrics import recall_score, precision_score, confusion_matrix, f1_score, precision_recall_curve,average_precision_score

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred, labels=None)
f1 = f1_score(y_test, y_pred)

print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("\nConfusion matrix: ")
print("\n   S    -S")
print(" S TP   FN")
print("-S FP   TN")
print(confusion)
print("\nF1 score: "+str(f1))

Precision: 0.679425837321
Recall: 0.804532577904

Confusion matrix: 

   S    -S
 S TP   FN
-S FP   TN
[[256 134]
 [ 69 284]]

F1 score: 0.736705577173


In [283]:
import matplotlib.pyplot as plt

precision_curve, recall_curve, thresholds = precision_recall_curve(y_test, y_pred)
precision_plot, recall_plot,_ = precision_recall_curve(y_test.ravel(), y_pred.ravel())
print(precision_curve, recall_curve, thresholds)
average_precision = average_precision_score(y_test.ravel(), y_pred.ravel())
print(average_precision)

plt.clf()
plt.plot(recall_plot, precision_plot, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall example: AUC={0:0.2f}'.format(average_precision))
plt.legend(loc="lower left")
plt.show()

[ 0.50336474  0.74045802  1.        ] [ 1.          0.77807487  0.        ] [0 1]
0.815121084126
